In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sympy as sp
from itertools import product

from Init import init_S, init_R, init_G
from Groups import GroupOhD
from Functions import *
from Interpolators import momenta_1p,  momenta_2p, interpolator_1p, interpolator_2p, computeR


#====constants

IRREPS  = {'2Oh' : ['G1g','G1u','Hg','Hu'],
           '2C4v': ['G1','G2'],
           '2C2v': ['G'],
           '2C3v': ['G','F1','F2']}
IRREPS  = {'2Oh' : ['G1g','G1u','Hg','Hu'],
           '2C4v': ['G1','G2'],
           '2C2v': [],
           '2C3v': []}
PREFS   = {'2Oh' : (0,0,0), 
           '2C4v': list(init_G['2C4v'].keys())[0], #0,0,1
           '2C2v': list(init_G['2C2v'].keys())[0], #0,1,1
           '2C3v': list(init_G['2C3v'].keys())[0],}#1,1,1

#====Matrix representations

#initialize group
group_2Oh = GroupOhD()

#spin1212 matrix
S = extract_repr(group_2Oh,init_S)

#spin1 matrix
Rot3d = extract_repr(group_2Oh,init_R)

#==== Irrep matrix

#irreps 2Oh
irrep_2Oh  = { I : extract_repr(group_2Oh,init_G['2Oh'][I]) for I in IRREPS['2Oh']}

#irreps 2C4v
group_2C4v = group_2Oh.little_group(Rot3d,sp.Matrix(PREFS['2C4v']))
irrep_2C4v = { I : extract_repr(group_2C4v,init_G['2C4v'][PREFS['2C4v']][I]) for I in IRREPS['2C4v']}

#irreps 2C3v
group_2C3v = group_2Oh.little_group(Rot3d,sp.Matrix(PREFS['2C3v']))
irrep_2C3v = { I : extract_repr(group_2C3v,init_G['2C3v'][PREFS['2C3v']][I]) for I in IRREPS['2C3v']}

#irreps 2C2v
group_2C2v = group_2Oh.little_group(Rot3d,sp.Matrix(PREFS['2C2v']))
irrep_2C2v = { I : extract_repr(group_2C2v,init_G['2C2v'][PREFS['2C2v']][I]) for I in IRREPS['2C2v']}

# other_PREFS = { G:compute_otherPref( PREFS[G], Rot3d ) for G in ['2C4v','2C3v','2C2v']}
# other_PREFS['2Oh'] = {}
other_PREFS = { '2Oh' :{},
               '2C4v':{(0,0,-1):'C_2a',
                       (0,-1,0):'C_4x',
                       (0,1,0):'C_4x^i',
                       (1,0,0) :'C_4y',
                       (-1,0,0) :'C_4y^i', },
              '2C2v':{(0,1,-1):'C_4x^i',
                      (0,-1,1):'C_4x',
                      (0,-1,-1):'C_2f',
                      
                      (1,0,1) :'C_4z^i',
                      (-1,0,-1):'C_2b',
                      (1,0,-1):'C_2a',
                      (-1,0,1) :'C_4z',
                      
                      (1,1,0) :'C_4y',
                      (-1,-1,0) :'C_2d',
                      (1,-1,0):'C_2c',
                      (-1,1,0):'C_4y^i'},
              '2C3v':{(1,1,-1) :'C_4y',
                     (1,-1,1) :'C_4x',
                     (-1,1,1):'C_4y^i',
                     (1,-1,-1):'C_2x',
                     (-1,1,-1):'C_2y',
                     (-1,-1,1):'C_2z',
                     (-1,-1,-1):'C_2b',
                     }}

other_PREFS = { '2Oh' :{},
               '2C4v':{(0,0,-1):'C_2y',
                       (0,-1,0):'C_4x',
                       (0,1,0):'C_4x^i',
                       (1,0,0) :'C_4y',
                       (-1,0,0) :'C_4y^i', },
               }

GROUPS      = { '2Oh':group_2Oh, '2C4v':group_2C4v, '2C3v':group_2C3v, '2C2v':group_2C2v }
irrepsG     = { '2Oh':irrep_2Oh, '2C4v':irrep_2C4v, '2C3v':irrep_2C3v, '2C2v':irrep_2C2v }

## Interpolators

* Delta interpolator:
$$\text{snk}\qquad\Delta^i_\gamma(x) \propto \epsilon_{abc}\;\;q^{1,T}_{a\alpha}(x)\: (\Gamma^{\Delta}_i)_{\alpha\beta}\: q^2_{b\beta}(x)\;\; q^3_{c\gamma}(x) $$
$$\text{src}\qquad\bar{\Delta}^i_\gamma(x) \propto \epsilon_{abc}\;\;\bar{q}^{2,T}_{a\alpha}(x)\: (\bar{\Gamma}^{\Delta}_i)_{\alpha\beta}\: \bar{q}^1_{b\beta}(x)\;\; \bar{q}^3_{c\gamma}(x) $$

* Nucleon interpolator:
$$\text{snk}\qquad N_\gamma(x) \propto \epsilon_{abc}\;\;q^{1,T}_{a\alpha}(x)\: (\Gamma^{N})_{\alpha\beta}\: q^2_{b\beta}(x)\;\; q^3_{c\gamma}(x) $$
$$\text{src}\qquad\bar{N}_\gamma(x) \propto \epsilon_{abc}\;\;\bar{q}^{2,T}_{a\alpha}(x)\: (\bar{\Gamma}^{N})_{\alpha\beta}\: \bar{q}^1_{b\beta}(x)\;\; \bar{q}^3_{c\gamma}(x) $$

* Meson interpolator:
$$\text{snk}\qquad\Pi(x) \propto \bar{q}^{1}_{a\alpha}(x)\: (\Gamma^{\Pi})_{\alpha\beta}\: q^2_{a\beta}(x) $$
$$\text{src}\qquad\bar{\Pi}(x) \propto \bar{q}^{2}_{a\alpha}(x)\: (\bar{\Gamma}^{\Pi})_{\alpha\beta}\: q^1_{a\beta}(x) $$
up to minus signs coming from the adjoint, and where $\bar{\Gamma}=\gamma_4\Gamma^{\dagger}\gamma_4$

In [8]:
key='C_4z'
# key='I_sC_2y'
print(Rot3d.keys())
Rot3d[key]
S[key]


dict_keys(['C_4y', 'C_4z', 'I_sE', 'C_2y', 'C_3delta', 'I_sC_4y', 'C_3gamma^i', 'C_2z', 'I_sC_4z', 'E', '\\bar{C}_4y^i', 'C_2a', 'I_sC_2y', 'C_2e', 'C_2c', 'I_sC_3delta', '\\bar{C}_2b', 'I_sC_3gamma^i', 'C_2d', '\\bar{C}_4z^i', 'I_sC_2z', '\\bar{E}', '\\bar{C}_3alfa', 'I_s\\bar{C}_4y^i', '\\bar{C}_3delta^i', 'C_2x', 'I_sC_2a', '\\bar{C}_3gamma', 'I_sC_2e', '\\bar{C}_3beta', 'I_sC_2c', '\\bar{C}_3beta^i', 'I_s\\bar{C}_2b', '\\bar{C}_2x', 'I_sC_2d', '\\bar{C}_3alfa^i', 'I_s\\bar{C}_4z^i', '\\bar{C}_4y', '\\bar{C}_4z', 'I_s\\bar{E}', '\\bar{C}_4x^i', '\\bar{C}_2d', 'I_s\\bar{C}_3alfa', 'I_s\\bar{C}_3delta^i', 'C_2b', 'I_sC_2x', '\\bar{C}_4x', 'I_s\\bar{C}_3gamma', '\\bar{C}_2f', 'I_s\\bar{C}_3beta', 'C_2f', 'I_s\\bar{C}_3beta^i', '\\bar{C}_2c', 'I_s\\bar{C}_2x', '\\bar{C}_2a', 'I_s\\bar{C}_3alfa^i', '\\bar{C}_2y', '\\bar{C}_3delta', 'I_s\\bar{C}_4y', '\\bar{C}_3gamma^i', '\\bar{C}_2z', 'I_s\\bar{C}_4z', 'I_s\\bar{C}_4x^i', 'C_3gamma', 'I_s\\bar{C}_2d', 'C_3beta^i', 'I_sC_2b', 'I_s\\bar{C}

Matrix([
[sqrt(2)/2 - sqrt(2)*I/2,                       0,                       0,                       0],
[                      0, sqrt(2)/2 + sqrt(2)*I/2,                       0,                       0],
[                      0,                       0, sqrt(2)/2 - sqrt(2)*I/2,                       0],
[                      0,                       0,                       0, sqrt(2)/2 + sqrt(2)*I/2]])

In [2]:
#functions for describing how the spin1212 rep acts on \Gamma of the interpolator.
u_diquark = lambda x,y: (y*x*y.T).applyfunc(sp.nsimplify)     # y = S^-1 if snk, S^T if src
u_meson   = lambda x,y: ((y.T**-1)*x*y.T).applyfunc(sp.nsimplify) # y = S^-1 
# Yan: I don't think this is correct, but it is trivial for meson and nucleon cases. 

#interpolators at snk
list_interpsD  = list_interps(['cg1','cg2','cg3'])
list_interpsN  = list_interps(['cg5'])
list_interpsP  = list_interps(['g5'])

## Compute coefficients

In [3]:
def do_1p(name,interps, addtodict=None):
    proj_N, coeff_N = rec_dd(), rec_dd()
    output   = copy_dict(addtodict) if addtodict is not None else rec_dd()
    printout = []
    for LG,Ilist in IRREPS.items():
        print(LG,Ilist)
        # if LG != '2Oh':
        #     continue
        #p_ref
        p0 = PREFS[LG]

        #define 1p interpolator obj
        nucleon = interpolator_1p(name,[list(p0),],'gs',interps)

        #define U matrix for Nucleon
        UN = { R:nucleon.Wt(Rot3d[R],S[R]**-1,f_i=u_diquark) for R in GROUPS[LG].list_elements() }
        
        #For  all the irreps
        for I in Ilist:
            # if I !='G1g':
            #     continue
            #compute coeff for p_ref
            proj_N[LG][I][p0], coeff_N[LG][I][p0] = computeProjs(irrepsG[LG][I],UN,0)
            #if zero, i.e. this irrep does not appear 
            if coeff_N[LG][I][p0]['l1']==sp.zeros(0,0):
                del coeff_N[LG][I][p0]['l1']
                del coeff_N[LG][I][p0]
                del coeff_N[LG][I]
                continue
            #write in output for p_ref
            for l in coeff_N[LG][I][p0]: 
                nsize = coeff_N[LG][I][p0][l].shape[0]
                for n in range(nsize):
                    repl    = chr(ord('`')+n+1)
                    aux_str = 'out[%s][%s][%s][%s][%s]'%(LG,I,l,p0,repl)
                    l_arr, l_str = add_out(coeff_N[LG][I][p0][l].row(n), interps['list'])
                    for k,v in l_arr.items():
                        output[LG][I][l][p0][repl][k]=v
                    for s in l_str:
                        printout.append(aux_str+s)
                        
            #compute coeffs for other p_refs
            for p,Rprime in other_PREFS[LG].items():
                psprime, UN_Rpr = nucleon.apply_outR( Rot3d[Rprime], S[Rprime]**-1, u_diquark )
                #for each row
                for r in coeff_N[LG][I][p0]:
                    #not sure about that!!
                    coeff_N[LG][I][p][r] = coeff_N[LG][I][p0][r]*UN_Rpr

                    #write in output for p_ref
                    nsize = coeff_N[LG][I][p][r].shape[0]
                    for n in range(nsize):
                        repl    = chr(ord('`')+n+1)
                        aux_str = 'out[%s][%s][%s][%s][%s]'%(LG,I,r,p,repl)
                        l_arr, l_str = add_out(coeff_N[LG][I][p][r].row(n), interps['list'])
                        for k,v in l_arr.items():
                            output[LG][I][r][p][repl][k]=v
                        for s in l_str:
                            printout.append(aux_str+s)
           
    proj_N  = proj_N.todict()
    coeff_N = coeff_N.todict()
    if addtodict is None:
        output = output.todict()
    return proj_N, coeff_N, output, printout

proj_N, coeff_N, ctosave, printout = do_1p('N',list_interpsN)
#same also for ['cg5g4']
ctosave_1 = dict_repl(ctosave,{'cg5':'cg5g4'})
ctosave_m = merge_dicts(ctosave,ctosave_1)

write_pickle(ctosave_m,'coeffsN.pkl')

2Oh ['G1g', 'G1u', 'Hg', 'Hu']
2C4v ['G1', 'G2']
2C2v []
2C3v []


In [4]:
avgDirection={}
for gp in other_PREFS.keys():
    # pref=','.join([str(ele) for ele in PREFS[gp]])
    # print(pref)
    for p,rot in other_PREFS[gp].items():
        # print(gp,p,rot)
        pt=','.join([str(ele) for ele in p])
        avgDirection[pt]={'rot_ele':rot,'irrep_row':np.array(irrep_2Oh['G1g'][rot]**-1,dtype=complex),'gamma_i':np.array(Rot3d[rot]**-1,dtype=complex),'spin':np.array(S[rot]**-1,dtype=complex)}

gt=group_2Oh.group_table()
def mul(ele1,ele2):
    return gt[ele1][ele2]
def inv(ele):
    for inv in gt[ele].keys():
        if gt[ele][inv]=='E':
            return inv

avgDirection['P']={}
for p,pP in [('0,0,1','0,0,-1'),('0,1,0','0,-1,0'),('1,0,0','-1,0,0')]:
    rot=avgDirection[p]['rot_ele'] if p!='0,0,1' else 'E'
    rotP=avgDirection[pP]['rot_ele']
    
    b=mul(rot,mul(inv(rotP),'I_sE'))
    bP=mul(rotP,mul(inv(rot),'I_sE'))

    b=np.array(irrep_2C4v['G1'][b]**-1,dtype=complex)
    bP=np.array(irrep_2C4v['G1'][bP]**-1,dtype=complex)
    # print(p)
    # print(b)
    # print()
    # print(pP)
    # print(bP)
    # print()
    
    assert(b[0,0]==b[1,1]==bP[0,0]==bP[1,1]==0)
    
    avgDirection['P'][p]={'l1':b[0,1],'l2':b[1,0]}
    avgDirection['P'][pP]={'l1':bP[0,1],'l2':bP[1,0]}

import pickle
with open('avgDirection.pkl','wb') as f:
    pickle.dump(avgDirection,f)

# avgDirection

In [6]:
def do_2p(names,interps, addtodict=None ):
    proj_2p, coeff_2p = rec_dd(), rec_dd()
    output   = copy_dict(addtodict) if addtodict is not None else rec_dd()
    printout = []
    
    for LG,Ilist in IRREPS.items():
        # if LG != '2C3v':
        #     continue
        #p_ref
        p0 = PREFS[LG]

        mom_lists = { 'N%d\pi%d'%(i,j) : momenta_2p(p0,i,j) for i,j in product(range(4),range(4)) 
                    if len(momenta_2p(p0,i,j)[0])!=0 }
        for labNP in mom_lists:
            # if labNP != 'N2\pi1':
            #     continue
            #define 2p interpolator obj
            Npi = interpolator_2p(names,mom_lists[labNP],['gs','g'],interps)
           
            #define U matrix for Npi basis
            UNP = { R:Npi.Wt(Rot3d[R],S[R]**(-1),u_diquark,u_meson) for R in GROUPS[LG].list_elements() }

            #For  all the irreps
            for I in Ilist:
                # if I != 'F1':
                #     continue
                print(LG,' -- ',I,'\tp: ',labNP)
                #compute coeff for p_ref
                proj_2p[LG][I][p0][labNP], coeff_2p[LG][I][p0][labNP] = computeProjs(irrepsG[LG][I],UNP,0)
                                
                #if zero, i.e. this irrep does not appear 
                if coeff_2p[LG][I][p0][labNP]['l1']==sp.zeros(0,0):
                    del coeff_2p[LG][I][p0][labNP]['l1']
                    del coeff_2p[LG][I][p0][labNP]
                    if len(coeff_2p[LG][I][p0])==0:
                        del coeff_2p[LG][I][p0]
                        del coeff_2p[LG][I]
                    continue
                #write in output for p_ref
                for l in coeff_2p[LG][I][p0][labNP]: 
                    nsize = coeff_2p[LG][I][p0][labNP][l].shape[0]
                    for n in range(nsize):
                        repl    = chr(ord('`')+n+1)
                        aux_str = 'out[%s][%s][%s][%s][%s][%s]'%(LG,I,l,p0,labNP,repl)
                        l_arr, l_str = add_out2p(coeff_2p[LG][I][p0][labNP][l].row(n), Npi)
                        for mom,tmp in l_arr.items():
                            for g,v in tmp.items():
                                output[LG][I][l][p0][labNP][repl][mom][g]=v
                        for s in l_str:
                            printout.append(aux_str+s)
                
                #compute coeffs for other p_refs
                for p,Rprime in other_PREFS[LG].items():
                    psprime, UNP_Rpr = Npi.apply_outR( Rot3d[Rprime], S[Rprime]**(-1), u_diquark, u_meson )
                    rotNpi = Npi.rotate_moms(Rot3d[Rprime])
                    #for each row
                    for r in coeff_2p[LG][I][p0][labNP]:
                        #not sure about that!!
                        coeff_2p[LG][I][p][labNP][r] = coeff_2p[LG][I][p0][labNP][r]*UNP_Rpr

                        #write in output 
                        nsize = coeff_2p[LG][I][p][labNP][r].shape[0]
                        for n in range(nsize):
                            repl    = chr(ord('`')+n+1)
                            aux_str = 'out[%s][%s][%s][%s][%s][%s]'%(LG,I,r,p,labNP,repl)
                            l_arr, l_str = add_out2p(coeff_2p[LG][I][p][labNP][r].row(n), rotNpi)
                            for mom,tmp in l_arr.items():
                                for g,v in tmp.items():
                                    output[LG][I][r][p][labNP][repl][mom][g]=v
                            for s in l_str:
                                printout.append(aux_str+s)

    proj_2p  = proj_2p.todict()
    coeff_2p = coeff_2p.todict()
    if addtodict is None:
        output = output.todict()
    return proj_2p, coeff_2p, output, printout

proj_Npi, coeff_Npi, ctosave, printout = do_2p( ['N','\pi'], [list_interpsN,list_interpsP] )
#same also for ['cg5g4']
ctosave_1 = dict_repl(ctosave,{'cg5;g5':'cg5g4;g5'})
ctosave_m = merge_dicts(ctosave,ctosave_1)

write_pickle(ctosave_m,'coeffsNpi.pkl')

<>:12: SyntaxWarning: invalid escape sequence '\p'
<>:12: SyntaxWarning: invalid escape sequence '\p'
C:\Users\yan14\AppData\Local\Temp\ipykernel_400\1845408211.py:12: SyntaxWarning: invalid escape sequence '\p'
  mom_lists = { 'N%d\pi%d'%(i,j) : momenta_2p(p0,i,j) for i,j in product(range(4),range(4))


2Oh  --  G1g \tp:  N0\pi0
2Oh  --  G1u \tp:  N0\pi0
2Oh  --  Hg \tp:  N0\pi0
2Oh  --  Hu \tp:  N0\pi0
2Oh  --  G1g \tp:  N1\pi1
2Oh  --  G1u \tp:  N1\pi1
2Oh  --  Hg \tp:  N1\pi1
2Oh  --  Hu \tp:  N1\pi1
2Oh  --  G1g \tp:  N2\pi2
2Oh  --  G1u \tp:  N2\pi2
2Oh  --  Hg \tp:  N2\pi2
2Oh  --  Hu \tp:  N2\pi2


C:\Users\yan14\AppData\Local\Temp\ipykernel_400\1845408211.py:12: SyntaxWarning: invalid escape sequence '\p'
  mom_lists = { 'N%d\pi%d'%(i,j) : momenta_2p(p0,i,j) for i,j in product(range(4),range(4))


KeyboardInterrupt: 

In [ ]:
# test
LG = '2Oh'
I  = 'G1g'
p0 = (0,0,0)

mom_list = momenta_2p(p0,1,1) 

Npi   = interpolator_2p(['N','\\pi'],mom_list,['gs','g'],[list_interpsN,list_interpsP])
delta = interpolator_1p('\Delta',[p0,],'gs',list_interpsD)
            
    
#define U matrix for Npi basis
UNP = { R:Npi.Wt(Rot3d[R],S[R]**(-1),u_diquark,u_meson) for R in GROUPS[LG].list_elements() }
UD  = { R:delta.Wt(Rot3d[R],S[R]**(-1),f_i=u_diquark) for R in GROUPS[LG].list_elements() }

#projs, coeff = computeProjs(irrepsG[LG][I],UD,0)
projs, coeff = computeProjs(irrepsG[LG][I],UNP,0)

In [ ]:
Npi.components;

In [ ]:
coeff['l1']

In [ ]:
projs['l1']

## Do nucleon

In [23]:
proj_N, coeff_N, ctosave, printout = do_1p('N',list_interpsN)
#same also for ['cg5g4']
ctosave_1 = dict_repl(ctosave,{'cg5':'cg5g4'})
ctosave_m = merge_dicts(ctosave,ctosave_1)

In [24]:
write_pickle(ctosave_m,'coeffsN.pkl')

In [25]:
with open('coeffsNucleon.txt','w') as writer:
    writer.write("#out[Irrep][Row][Ptot][replica][interpolator][spin] = coeff\n")
    for line in printout:
        if not complex(line.split('=')[1])==0+1j*0:
            writer.write(line+'\n')

## Do Delta

In [26]:
proj_D, coeff_D, ctosave, printout = do_1p('\Delta',list_interpsD)
#same also for ['cg1g4g5-g5','cg2g4g5-g5','cg3g4g5-g5']
ctosave_1 = dict_repl(ctosave,{'cg1':'cg1g4g5-g5','cg2':'cg2g4g5-g5','cg3':'cg3g4g5-g5'})
ctosave_m = merge_dicts(ctosave,ctosave_1)

In [27]:
write_pickle(ctosave_m,'coeffsDelta.pkl')

In [28]:
with open('coeffsDelta.txt','w') as writer:
    writer.write("#out[Irrep][Row][Ptot][replica][interpolator][spin] = coeff\n")
    for line in printout:
        if not complex(line.split('=')[1])==0+1j*0:
            writer.write(line+'\n')

## Do piN

In [29]:
proj_Npi, coeff_Npi, ctosave, printout = do_2p( ['N','\pi'], [list_interpsN,list_interpsP] )
#same also for ['cg5g4']
ctosave_1 = dict_repl(ctosave,{'cg5;g5':'cg5g4;g5'})
ctosave_m = merge_dicts(ctosave,ctosave_1)

2Oh  --  G1g 	p:  N0\pi0
2Oh  --  G1u 	p:  N0\pi0
2Oh  --  Hg 	p:  N0\pi0
2Oh  --  Hu 	p:  N0\pi0
2Oh  --  G1g 	p:  N1\pi1
2Oh  --  G1u 	p:  N1\pi1
2Oh  --  Hg 	p:  N1\pi1
2Oh  --  Hu 	p:  N1\pi1
2Oh  --  G1g 	p:  N2\pi2
2Oh  --  G1u 	p:  N2\pi2
2Oh  --  Hg 	p:  N2\pi2
2Oh  --  Hu 	p:  N2\pi2
2Oh  --  G1g 	p:  N3\pi3
2Oh  --  G1u 	p:  N3\pi3
2Oh  --  Hg 	p:  N3\pi3
2Oh  --  Hu 	p:  N3\pi3
2C4v  --  G1 	p:  N0\pi1
2C4v  --  G2 	p:  N0\pi1
2C4v  --  G1 	p:  N1\pi0
2C4v  --  G2 	p:  N1\pi0
2C4v  --  G1 	p:  N1\pi2
2C4v  --  G2 	p:  N1\pi2
2C4v  --  G1 	p:  N2\pi1
2C4v  --  G2 	p:  N2\pi1
2C4v  --  G1 	p:  N2\pi3
2C4v  --  G2 	p:  N2\pi3
2C4v  --  G1 	p:  N3\pi2
2C4v  --  G2 	p:  N3\pi2
2C2v  --  G 	p:  N0\pi2
2C2v  --  G 	p:  N1\pi1
2C2v  --  G 	p:  N1\pi3
2C2v  --  G 	p:  N2\pi0
2C2v  --  G 	p:  N2\pi2
2C2v  --  G 	p:  N3\pi1
2C3v  --  G 	p:  N0\pi3
2C3v  --  F1 	p:  N0\pi3
2C3v  --  F2 	p:  N0\pi3
2C3v  --  G 	p:  N1\pi2
2C3v  --  F1 	p:  N1\pi2
2C3v  --  F2 	p:  N1\pi2
2C3v  --  G 	p: 

In [30]:
write_pickle(ctosave_m,'coeffsNpi.pkl')

In [32]:
with open('coeffsNpi.txt','w') as writer:
    writer.write("#out[Irrep][Row][Ptot][N<|p_N|^2>P<|p_pi|^2>][replica][p_N,p_pi][interpolator][spin] = coeff\n")
    for line in printout:
        if not complex(line.split('=')[1])==0+1j*0:
            writer.write(line+'\n')